<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/jihyun_llama_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Settings

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install peft

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

2. Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Add the padding token to the tokenizer
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
# If you are using the model for training, also resize the model embeddings to include the padding token
model.resize_token_embeddings(len(tokenizer))


3. Dataset load, 전처리

In [ ]:
import os
from datasets import load_dataset
import kagglehub

# Kaggle에서 Stack Overflow 데이터셋 다운로드
path = kagglehub.dataset_download("imoore/60k-stack-overflow-questions-with-quality-rate")

# train.csv 파일 찾기
train_file = None
files = os.listdir(path)
for file in files:
    if 'train.csv' in file:
        train_file = os.path.join(path, file)
        break

# 데이터셋 로드
if train_file:
    train_dataset = load_dataset('csv', data_files=train_file)
else:
    print("train.csv 파일을 찾을 수 없습니다.")

# 'body'와 'tags' 열 추출
def preprocess_function(examples):
    return {'input_text': examples['Body'], 'labels': examples['Tags']}

train_dataset = train_dataset['train'].map(preprocess_function)


4. Tokenizing

5. tran data loader

6. Train

7. Model 학습이 끝난 모델 저장

In [ ]:
# 학습이 끝나면 모델과 토크나이저 저장
model.save_pretrained("./fine_tuned_llama")
tokenizer.save_pretrained("./fine_tuned_llama")


8. 모델 평가